<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/embedding_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

references:

https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go

https://www.kaggle.com/alhalimi/tokenization-and-word-embedding-compatibility

https://www.kaggle.com/canming/ensemble-mean-iii-64-36

In [ ]:
import gc

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
import os

# input_path = '/kaggle/input/quora-insincere-questions-classification'
input_path = '/content/drive/MyDrive/ColabNotebooks/QIQC/data'
train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [ ]:
if os.path.exists(train_path): train_df = pd.read_csv(train_path)
else: train_df = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test_df = pd.read_csv(train_path)
else: test_df = pd.read_csv(test_path + '.zip')

print('train shape:', train_df.shape)
print('test shape:', test_df.shape)

train shape: (1306122, 3)
test shape: (1306122, 3)


In [ ]:
from sklearn.model_selection import train_test_split

# split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1)

# some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

# fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

# Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [ ]:
%%time
from keras.preprocessing.text import Tokenizer

# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

CPU times: user 50.2 s, sys: 508 ms, total: 50.7 s
Wall time: 50.6 s


In [ ]:
%%time
from keras.preprocessing.sequence import pad_sequences

# Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

CPU times: user 10.3 s, sys: 304 ms, total: 10.6 s
Wall time: 10.4 s


In [ ]:
import zipfile
from gensim.models import KeyedVectors

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
para = 'paragram_300_sl999/paragram_300_sl999.txt'


def load_embedding(embedding_name):

    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

    with zipfile.ZipFile(embeddings_path) as embeddings_zip:
        print("Found embeddings as a zip file")

        if embedding_name == google:
            return KeyedVectors.load_word2vec_format(embeddings_zip.open(google), binary=True)

        else:
            embedding = []
            for o in embeddings_zip.open(embedding_name):
                try:
                    if len(o.decode('utf-8')) > 100:
                        embedding.append(get_coefs(*o.decode('utf-8').split(" ")))
                except:
                    pass
        
        return dict(embedding)

In [ ]:
%%time
glove_embedding = load_embedding(glove) # 3min 21s in Colab

Found embeddings as a zip file
CPU times: user 3min 13s, sys: 3.82 s, total: 3min 17s
Wall time: 3min 18s


In [ ]:
%%time
wiki_embedding = load_embedding(wiki) # 1min 20s in Colab

Found embeddings as a zip file
CPU times: user 1min 19s, sys: 2.1 s, total: 1min 21s
Wall time: 1min 21s


In [ ]:
%%time
google_embedding = load_embedding(google) # 1min 50s in Colab

Found embeddings as a zip file
CPU times: user 1min 46s, sys: 3.65 s, total: 1min 50s
Wall time: 1min 50s


In [ ]:
%%time
para_embedding = load_embedding(para) # 2min 40s in Colab

Found embeddings as a zip file
CPU times: user 2min 34s, sys: 2.81 s, total: 2min 36s
Wall time: 2min 36s


In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

In [ ]:
has_get_emb_stats = True # set False if we have not get emb stats
if not has_get_emb_stats:

    def get_emb_stats(embedding):

        # Put all embeddings in a numpy matrix
        try:
            all_embs= np.stack(embedding.values())
        except:
            all_embs = embedding.vectors

        # Get embedding stats
        emb_mean = all_embs.mean()
        emb_std = all_embs.std()
        
        return emb_mean,emb_std
    

    glove_emb_mean, glove_emb_std = get_emb_stats(glove_embedding)
    wiki_emb_mean, wiki_emb_std = get_emb_stats(wiki_embedding)
    google_emb_mean, google_emb_std = get_emb_stats(google_embedding)
    para_emb_mean, para_emb_std = get_emb_stats(para_embedding)

    print('glove emb mean:', glove_emb_mean, 'emb_std:', glove_emb_std)
    print('wiki emb mean:', wiki_emb_mean, 'emb_std:', wiki_emb_std)
    print('google emb mean:', google_emb_mean, 'emb_std:', google_emb_std)
    print('para emb mean:', para_emb_mean, 'emb_std:', para_emb_std)

else:
    glove_emb_mean, glove_emb_std = -0.005838499, 0.48782197
    wiki_emb_mean, wiki_emb_std = -0.0033469985, 0.109855495
    google_emb_mean, google_emb_std = -0.003527845, 0.13315111
    para_emb_mean, para_emb_std = -0.0053248387, 0.49346521

In [ ]:
glove_embedding_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
wiki_embedding_matrix = np.random.normal(wiki_emb_mean, wiki_emb_std, (max_features, embed_size))
google_embedding_matrix = np.random.normal(google_emb_mean, google_emb_std, (max_features, embed_size))
para_embedding_matrix = np.random.normal(para_emb_mean, para_emb_std, (max_features, embed_size))

In [ ]:
glove_oov = {}
wiki_oov = {}
google_oov = {}
para_oov = {}
for word, i in tqdm(word_index.items()):
    if i >= max_features: continue
    if word in glove_embedding:
        embedding_vector = glove_embedding[word]
        glove_embedding_matrix[i] = embedding_vector
    else:
        glove_oov[word] = i
    if word in wiki_embedding:
        embedding_vector = wiki_embedding[word]
        wiki_embedding_matrix[i] = embedding_vector
    else:
        wiki_oov[word] = i
    if word in google_embedding:
        embedding_vector = google_embedding[word]
        google_embedding_matrix[i] = embedding_vector
    else:
        google_oov[word] = i
    if word in para_embedding:
        embedding_vector = para_embedding[word]
        para_embedding_matrix[i] = embedding_vector
    else:
        para_oov[word] = i

In [ ]:
print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))
print('percentage of oov of wiki: {:.2f}%'.format(len(wiki_oov) / max_features * 100))
print('percentage of oov of google: {:.2f}%'.format(len(google_oov) / max_features * 100))
print('percentage of oov of para: {:.2f}%'.format(len(para_oov) / max_features * 100))

percentage of oov of glove: 8.02%
percentage of oov of wiki: 14.56%
percentage of oov of google: 23.63%
percentage of oov of para: 5.55%


We will clean the numbers.

In [ ]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    
    return x

In [ ]:
no_num_tokenizer = Tokenizer()
no_num_tokenizer.fit_on_texts(list(train_df['question_text'].fillna('__na__').progress_apply(lambda x: clean_numbers(x)).values))

In [ ]:
import random

for i in range(10):
    n = random.randint(0, 20000)
    if str(n) not in glove_embedding: print(n, 'is not in glove embedding')
    if str(n) not in wiki_embedding: print(n, 'is not in wiki embedding')
    # if str(n) not in google_embedding: print(n, 'is not in google embedding')
    if str(n) not in para_embedding: print(n, 'is not in para embedding')

google embedding only have few numbers so we will clean all the numbers.

It seems like other embeddings have numbers up to 10,000 so we will not clean the numbers.

In [ ]:
no_num_word_index = no_num_tokenizer.word_index

google_oov = {}
for word, i in tqdm(no_num_word_index.items()):
    if i >= max_features: continue
    if word in google_embedding:
        embedding_vector = google_embedding[word]
        google_embedding_matrix[i] = embedding_vector
    else:
        google_oov[word] = i

In [ ]:
print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))
print('percentage of oov of wiki: {:.2f}%'.format(len(wiki_oov) / max_features * 100))
print('percentage of oov of google: {:.2f}%'.format(len(google_oov) / max_features * 100))
print('percentage of oov of para: {:.2f}%'.format(len(para_oov) / max_features * 100))

percentage of oov of google: 22.63% -> 22.13%

Some trials to make better performance..

In [ ]:
"don't" in google_embedding

In [ ]:
google_oov

In [ ]:
"i'm" in glove_oov

In [ ]:
glove_oov

In [ ]:
"you’re" in glove_embedding

In [ ]:
for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
    if punct not in glove_embedding:
        print(punct)

# Although “”’are in glove_embedding but we will replace it for convinience

In [ ]:
word = "someone’s"
word = word.replace("’", "'")
if word.endswith("'s") or word.endswith("' s"):
    w1, w2 = word.split("'")
print(w1)
print(w2)

In [ ]:
glove_oov = {}
for word, i in tqdm(word_index.items()):
    if i >= max_features: continue

    if word in glove_embedding:
        glove_embedding_matrix[i] = glove_embedding[word]
        continue

    word = word.replace("’", "'")
    word = word.replace('”', '"')
    word = word.replace('“', '"')
    if word in glove_embedding:
        glove_embedding_matrix[i] = glove_embedding[word]
        continue
    
    word = re.sub(' +', '', word)
    if word in glove_embedding:
        glove_embedding_matrix[i] = glove_embedding[word]
        continue

    tmp_word = word.split("'")[0]
    if tmp_word in glove_embedding:
        glove_embedding_matrix[i] = glove_embedding[tmp_word]
        continue
    
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~':
        word = word.replace(punct, '')
    if word in glove_embedding:
        glove_embedding_matrix[i] = glove_embedding[word]
        continue
    
    # quorans -> quora contributors
    

    else:
        glove_oov[word] = i

In [ ]:
print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))

8.15% -> 4.18%

In [ ]:
'Quora' in glove_embedding

In [ ]:
for k in word_index.keys():
    if word_index[k] == 49990:
        print(k)

In [ ]:
word_index["'call"]

In [ ]:
''

In [ ]:
glove_oov